In [1]:
import json
import os
import sys
from math import *
wd = os.path.abspath(os.getcwd()) # os.path.dirname(os.path.dirname(os.getcwd()))
wd = os.path.dirname(wd)
sys.path.append(wd+"\\src")
import numpy as np


In [2]:
from gaslayer import GasFluxCalculator, GasEOS, GasLayer,GridStrecher,Powder,PowderOvLayer

In [3]:
from tube import Tube

In [4]:
with open('gpowders.json') as f:
    all_powders = json.load(f)

In [5]:
all_powders[r'4\7']

{'name': '4\\7',
 'f': 1.027,
 'etta': 0.228,
 'alpha_k': 1.008,
 'T_1': 3006.0,
 'ro': 1.6,
 'I_k': 0.32,
 'Z_k': 1.488,
 'k_1': 0.811,
 'lambda_1': 0.081,
 'k_2': 0.505,
 'lambda_2': -1.024,
 'k_f': 0.0003,
 'k_l': 0.0016}

In [6]:
powder_eos = Powder.from_bd(all_powders[r'4\7'])

In [7]:
flux_calculator = GasFluxCalculator()
grid_strecher = GridStrecher()

In [23]:
d = 23 * 1e-3
tube = Tube([0,1], [d, d])

In [24]:
omega = 30 * 1e-3
q = 170 *1e-3
delta = 700
p_ign = 5e6
pf = 300E6
W_0 = omega/delta
x0 = 0
x1 = tube.get_x2(x0,W_0)
x1

0.10315200200386698

In [36]:
n_cells = 100
layer = PowderOvLayer(n_cells=n_cells, tube=tube, powder=powder_eos, flux_calculator=flux_calculator, grid_strecher=grid_strecher)
layer.xs_borders = np.linspace(x0, x1, n_cells+1)
def foo(x, *args):
    return delta, p_ign, 0, 0
layer.init_ropue_fromfoo(foo)
layer.init_taus_acustic()

In [37]:
def get_V(tau: float, lr: PowderOvLayer):
    v_right = lr.Vs_borders[-1]
    p_right = lr.ps[-1]
    x_right = lr.xs_borders[-1]
    if p_right > pf or abs(v_right) > 1e-3:
        a = lr.S[-1] * p_right / q
        return v_right + tau * a
    return v_right

In [38]:
courant = 0.5
layers_all = []
while layer.xs_borders[-1] < 1:
    tau = layer.get_tau_min() * courant
    vr = get_V(tau, layer)
    layer1 = layer.step_simple(tau, 0, vr)
    if layer == layer1:
        break
    layer = layer1

In [39]:
d = layer.to_dict()

In [42]:
print(d['Vs_borders'])

[  1.           8.87749541  16.75499082  24.63248623  32.50998164
  40.38747705  48.26497246  56.14246787  64.01996328  71.89745869
  79.7749541   87.65244951  95.52994492 103.40744033 111.28493574
 119.16243115 127.03992656 134.91742197 142.79491738 150.67241279
 158.5499082  166.42740361 174.30489902 182.18239443 190.05988984
 197.93738525 205.81488066 213.69237607 221.56987148 229.44736688
 237.32486229 245.2023577  253.07985311 260.95734852 268.83484393
 276.71233934 284.58983475 292.46733016 300.34482557 308.22232098
 316.09981639 323.9773118  331.85480721 339.73230262 347.60979803
 355.48729344 363.36478885 371.24228426 379.11977967 386.99727508
 394.87477049 402.7522659  410.62976131 418.50725672 426.38475213
 434.26224754 442.13974295 450.01723836 457.89473377 465.77222918
 473.64972459 481.52722    489.40471541 497.28221082 505.15970623
 513.03720164 520.91469705 528.79219246 536.66968787 544.54718328
 552.42467869 560.3021741  568.17966951 576.05716492 583.93466033
 591.81215